# Brain tumor detection model

### Adding imports

In [5]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split


### Loading Data

In [6]:
data_path = 'data/training'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

print(label_dict)
print(categories)
print(labels)

{'no': 0, 'yes': 1}
['no', 'yes']
[0, 1]


### Data preprocessing

In [7]:
img_size = 200
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)

    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)

        try:
            # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(img, (img_size, img_size))
            data.append(resized)
            target.append(label_dict[category])

        except Exception as e:
            print('Exception:', e)

### Saving preprocessed data

In [8]:
import warnings
warnings.filterwarnings('ignore')
data = np.array(data) / 255.0
data = np.reshape(data, (data.shape[0], img_size, img_size, 3))
target = np.array(target)

from keras.utils import np_utils
new_target = np_utils.to_categorical(target)

np.save('training/data', data)
np.save('training/target', new_target)

2024-10-12 14:40:17.380237: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load preprocessed data

In [9]:
data = np.load('training/data.npy')
target = np.load('training/target.npy')

### Model Architecture

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import keras.metrics as metrics

noOfFilters = 64
sizeOfFilter1 = (3, 3)
sizeOfFilter2 = (3, 3)
sizeOfPool = (2, 2)
noOfNodes = 128

model = Sequential()
model.add(
    (
        Conv2D(
            noOfFilters,
            sizeOfFilter1,
            input_shape=(img_size, img_size, 3),
            activation="relu",
        )
    )
)
model.add((Conv2D(noOfFilters, sizeOfFilter1, activation="relu")))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(noOfFilters, sizeOfFilter2, activation="relu")))
model.add((Conv2D(noOfFilters, sizeOfFilter2, activation="relu")))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add(Flatten())
model.add(Dense(noOfNodes, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(2, activation="softmax"))

model.compile(
    loss="categorical_crossentropy",
    optimizer="adamax",
    metrics=[metrics.Accuracy(), metrics.Precision(), metrics.Recall()],
)

In [11]:
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.1)

In [12]:
history = model.fit(train_data, train_target, epochs=30, validation_split=0.2, verbose=1)
model.save('saved_models/model1.keras')

Epoch 1/30
6/6 [==============================] - 20s 3s/step - loss: 1.0129 - precision: 0.6133 - recall: 0.6133 - accuracy: 0.0000e+00 - val_loss: 0.6177 - val_precision: 0.6304 - val_recall: 0.6304 - val_accuracy: 0.0000e+00
Epoch 2/30
3/6 [==============>...............] - ETA: 9s - loss: 0.6033 - precision: 0.6250 - recall: 0.6250 - accuracy: 0.0000e+00 

KeyboardInterrupt: 

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# import keras.metrics as metrics
# from keras.regularizers import l2

# noOfFilters = 64
# sizeOfFilter1 = (3, 3)
# sizeOfFilter2 = (3, 3)
# sizeOfPool = (2, 2)
# noOfNodes = 128

# model = Sequential()
# model.add((Conv2D(noOfFilters, sizeOfFilter1, input_shape=(img_size, img_size, 1), activation='relu', kernel_regularizer=l2(0.01))))
# model.add((Conv2D(noOfFilters, sizeOfFilter1, activation='relu')))
# model.add(MaxPooling2D(pool_size=sizeOfPool))

# model.add((Conv2D(noOfFilters, sizeOfFilter2, activation='relu')))
# model.add((Conv2D(noOfFilters, sizeOfFilter2, activation='relu')))
# model.add(MaxPooling2D(pool_size=sizeOfPool))
# model.add(Dropout(0.5))

# model.add(Flatten())
# model.add(Dense(noOfNodes, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(2, activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.Precision(),metrics.Recall(), metrics.Accuracy()])

# check_point = ModelCheckpoint('training/model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
# early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')